In [ ]:
from var4d_components import Var4D_Components
from visualize_results import Visualize_Obs, Visualize_Fluxes, Diagnostic_Plots

# Balancing data and prior uncertainty
Surface flux inversions are Bayesian estimates that trade off between our knowledge of fluxes prior to making atmospheric observations and our knowledge of the atmospheric state as reflected in observations. The former is represented by $\mathbf{S_x}^{-1}$ (smaller the uncertainty, "better" the knowledge) and the latter is represented by $\mathbf{S_z}^{-1}$. In this exercise, we will vary the balance between the two and see what the solution looks like.
## Set up an inversion to assimilate only NOAA surface observatories circa 1960
The surface sites of Mauna Loa (MLO) and South Pole (SPO) represent the longest atmospheric record from NOAA. As a base case, let's just assimilate those.

In [ ]:
var4d = Var4D_Components('explore_unc_balance', verbose=False, store_intermediate=True) # change verbose to False to see fancy progress bars and suppress prints
flux_corr_structure = {'temp_corr': 2.0} # 2-month temporal correlation, no horizontal correlation
obs_assim_dict = {'sites': ['mlo', 'spo']} # just two observatories

First, do the base case of NEE uncertainty = 0.25 $\times$ abs(NEE) and ocean flux uncertainty = 0.5 $\times$ abs(ocean flux)

In [ ]:
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 0.25, 'ocean': 0.5}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)

In [ ]:
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
po1 = Visualize_Obs('explore_unc_balance')
po1.plot_site(['spo','mlo','smo', 'brw'])

Now let's tighten the prior flux uncertainty by a factor of 10. *What do you expect?*

In [ ]:
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 0.025, 'ocean': 0.05}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)

In [ ]:
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
po1 = Visualize_Obs('explore_unc_balance')
po1.plot_site(['spo','mlo','smo', 'brw'])

# What does the adjoint forcing look like?

# Different ways of constructing $\mathbf{S_x}$